In [1]:
# install dependencies as necessary
!pip install -qr /Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/yolov5/requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
# from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Setup complete. Using torch 2.0.1 CPU


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("plasticdetection-kr0gi").project("plasticdetectioninrivers")
dataset = project.version(3).download("yolov5")




[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to plasticDetectionInRivers-3 in yolov5pytorch:: 100%|██████████| 232/232 [00:00<00:00, 2771.28it/s]


In [6]:
%cat /Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/plasticDetectionInRivers-3/data.yaml

names:
- plastic
nc: 1
roboflow:
  license: CC BY 4.0
  project: plasticdetectioninrivers
  url: https://universe.roboflow.com/plasticdetection-kr0gi/plasticdetectioninrivers/dataset/3
  version: 3
  workspace: plasticdetection-kr0gi
test: /Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/plasticDetectionInRivers-3/test
train: /Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/plasticDetectionInRivers-3/train
val: /Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/plasticDetectionInRivers-3/valid


# defining model config and architecture

In [5]:
# define number of classes based on YAML
import yaml
with open("/Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/plasticDetectionInRivers-3/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [7]:
num_classes

'1'

In [8]:
#this is the model configuration we will use
%cat /Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/yolov5/models/yolov5m.yaml

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.67  # model depth multiple
width_multiple: 0.75  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]

In [9]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [10]:
num_classes

'1'

In [11]:
%%writetemplate /Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/yolov5/models/custom_yolov5m.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 2  # model depth multiple
width_multiple: 1  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

# Train Custom Yolo v5 detector

In [12]:
# train yolov5s on custom data for 100 epochs
# time its performance
%time
%cd /Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/yolov5
!python train.py --img 640 --batch 20 --epochs 100 --data '/Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/plasticDetectionInRivers-3/data.yaml' --cfg ./models/custom_yolov5m.yaml --weights 'yolov5m.pt' --name yolov5m_results  --cache

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.1 µs
/Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/yolov5
train: weights=yolov5m.pt, cfg=./models/custom_yolov5m.yaml, data=/Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/plasticDetectionInRivers-3/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=20, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5m_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-224-g6262c7fe Python-3.10.9 torch-2.0.1 CPU

hyperparameters: lr0=0.01, 

# loading the trained model from roboflow

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace().project("plasticdetectioninrivers")
model = project.version(3).model


loading Roboflow workspace...
loading Roboflow project...


In [6]:
# infer on a local image
print(model.predict("/Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/plasticDetectionInRivers-3/train/images/DJI_0020_jpg.rf.68907455640f46deb12baea018577926.jpg", confidence=40, overlap=30).json())


{'predictions': [{'x': 417.0, 'y': 302.5, 'width': 14.0, 'height': 19.0, 'confidence': 0.7532166242599487, 'class': 'plastic', 'class_id': 0, 'image_path': '/Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/plasticDetectionInRivers-3/train/images/DJI_0020_jpg.rf.68907455640f46deb12baea018577926.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 196.5, 'y': 396.0, 'width': 19.0, 'height': 22.0, 'confidence': 0.7387670278549194, 'class': 'plastic', 'class_id': 0, 'image_path': '/Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/plasticDetectionInRivers-3/train/images/DJI_0020_jpg.rf.68907455640f46deb12baea018577926.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 284.0, 'y': 408.0, 'width': 16.0, 'height': 16.0, 'confidence': 0.44433990120887756, 'class': 'plastic', 'class_id': 0, 'image_path': '/Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/plasticDetectionInRivers-3/train/images/DJI_0020_jpg.rf.68907455640f46deb12baea018577926.jpg', 'pr

In [7]:
# visualize your prediction
model.predict("/Users/damodargupta/Downloads/AI Hackathon REVA University/Train/DJI_0071.jpg", confidence=40, overlap=30).save("/Users/damodargupta/Desktop/projects/PlasticDetectionInRivers/prediction.jpg")

# infer on an image hosted elsewhere
# print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())

In [8]:
# visualize your prediction
model.predict("/Users/damodargupta/Downloads/AI Hackathon REVA University/Train/DJI_0071.jpg")

# infer on an image hosted elsewhere
# print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())

In [1]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_geotagging_info(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    print(exif_data)
    geotagging_info = None
    for tag, value in exif_data.items():
        tag_name = TAGS.get(tag, tag)
        if tag_name == 'GPSInfo':
            geotagging_info = {GPSTAGS.get(t, t): v for t, v in value.items()}
    return geotagging_info

image_path = "/Users/damodargupta/Downloads/AI Hackathon REVA University/Train/DJI_0025.jpg"
geotag_info = get_geotagging_info(image_path)


{34853: {0: b'\x02\x03\x00\x00', 1: 'N', 2: (10.0, 53.759, 0.0), 3: 'E', 4: (106.0, 41.6141, 0.0), 5: b'\x00', 6: 38.762}, 296: 2, 34665: 212, 271: 'DJI', 272: 'FC300X', 305: 'Adobe Photoshop CC 2015 (Windows)', 274: 1, 306: '2021:05:20 16:25:09', 282: 240.0, 283: 240.0, 36864: b'0230', 37377: 10.287712, 37378: 2.970854, 36867: '2021:05:20 10:00:45', 36868: '2021:05:20 10:00:45', 37380: 0.0, 37381: 2.0, 37382: 0.0, 37383: 2, 37384: 0, 37385: 32, 37386: 3.61, 40961: 65535, 40962: 3992, 41989: 20, 41990: 0, 40963: 2992, 41992: 0, 41993: 0, 41994: 0, 41996: 0, 41728: b'\x03', 33434: 0.0008, 33437: 2.8, 41729: b'\x01', 34850: 8176, 41985: 0, 34855: 100, 41986: 1, 41987: 0, 42033: '2014031100', 42034: (20.7, 20.7, nan, nan), 42036: '20.7 mm', 41988: 0.0, 41991: 0}


In [2]:
geotag_info

{'GPSVersionID': b'\x02\x03\x00\x00',
 'GPSLatitudeRef': 'N',
 'GPSLatitude': (10.0, 53.759, 0.0),
 'GPSLongitudeRef': 'E',
 'GPSLongitude': (106.0, 41.6141, 0.0),
 'GPSAltitudeRef': b'\x00',
 'GPSAltitude': 38.762}